In [1]:
import os
import pandas as pd
import json

from common.utils import RESULTS_DIR, filename_to_obj

In [2]:
def results_as_pandas(filename):
    path = os.path.join(RESULTS_DIR, filename)
    with open(path, "r") as f:
        data = f.readlines()
    data = [json.loads(d) for d in data]
    data = pd.DataFrame(data)

    params = filename_to_obj(filename)
    print(params)
    for k, v in params.items():
        data[k] = v

    data = data.explode("evaluations")
    data = data.rename_axis("question_idx").reset_index()

    data = pd.concat([data, data["evaluations"].apply(pd.Series)], axis=1)
    evaluation_keys = data["evaluations"].apply(pd.Series).columns
    for col in evaluation_keys:
        data = pd.concat([data, data[col].apply(pd.Series).add_prefix(f"{col}/")], axis=1)
        data = data.drop(columns=col)
    data = data.drop(columns=["evaluations"])

    return data


results = os.listdir(RESULTS_DIR)
params_names = list(filename_to_obj(results[0]).keys())
results = pd.concat([results_as_pandas(r) for r in results])
results.head()

{'llm': 'mixtral-8x7b-instruct-v0.1.Q8_0', 'prompt_id': '1', 'temperature': '0.1', 'nli': 't5_xxl_true_nli_mixture', 'nlg': 'mistral-7b-instruct-v0.2.Q8_0'}
{'llm': 'mistral-7b-instruct-v0.2.Q8_0', 'prompt_id': '2', 'temperature': '0.1', 'nli': 't5_xxl_true_nli_mixture', 'nlg': 'mistral-7b-instruct-v0.2.Q8_0'}
{'llm': 'mistral-7b-instruct-v0.2.Q8_0', 'prompt_id': '1', 'temperature': '0.1', 'nli': 't5_xxl_true_nli_mixture', 'nlg': 'mistral-7b-instruct-v0.2.Q8_0'}
{'llm': 'mistral-7b-instruct-v0.2.Q8_0', 'prompt_id': '3', 'temperature': '0.1', 'nli': 't5_xxl_true_nli_mixture', 'nlg': 'mistral-7b-instruct-v0.2.Q8_0'}
{'llm': 'gpt-3.5-turbo-0125', 'prompt_id': '1', 'temperature': '0.1', 'nli': 't5_xxl_true_nli_mixture', 'nlg': 'mistral-7b-instruct-v0.2.Q8_0'}


,question_idx,question_id,llm,prompt_id,temperature,nli,nlg,citations/ais_recall,citations/ais_precision,citations/n_sentences,...,citations/sentences,citations/supported,citations/citations,citations/correct_citations,citations/out_of_range,correctness/answer_overlap,correctness/answer_entail,correctness/citations_recall,quality/answer_relevance,id
0,0,5abab42e55429955dce3eed2,mixtral-8x7b-instruct-v0.1.Q8_0,1,0.1,t5_xxl_true_nli_mixture,mistral-7b-instruct-v0.2.Q8_0,0.142857,1.0,7,...,"[Malcolm Subban, a Canadian ice hockey goalten...","[0, 0, 1, 0, 0, 0, 0]","[[], [], [4], [], [2], [], []]","[[], [], [True], [], [True], [], []]","[0, 0, 0, 0, 0, 0, 0]",1.0,0.0,0.5,0.403033,NaN
1,0,5abab42e55429955dce3eed2,mixtral-8x7b-instruct-v0.1.Q8_0,1,0.1,t5_xxl_true_nli_mixture,mistral-7b-instruct-v0.2.Q8_0,0.200000,1.0,5,...,"[Malcolm Subban, a Canadian ice hockey goalten...","[0, 1, 0, 0, 0]","[[], [4], [3], [], []]","[[], [True], [True], [], []]","[0, 0, 0, 0, 0]",1.0,0.0,1.0,0.549066,NaN
2,0,5abab42e55429955dce3eed2,mixtral-8x7b-instruct-v0.1.Q8_0,1,0.1,t5_xxl_true_nli_mixture,mistral-7b-instruct-v0.2.Q8_0,0.000000,1.0,4,...,[The professional hockey player related to P.K...,"[0, 0, 0, 0]","[[], [4], [4], [3]]","[[], [True], [True], [True]]","[0, 0, 0, 0]",1.0,1.0,1.0,0.707082,NaN
3,1,5a761900554299109176e648,mixtral-8x7b-instruct-v0.1.Q8_0,1,0.1,t5_xxl_true_nli_mixture,mistral-7b-instruct-v0.2.Q8_0,1.000000,1.0,1,...,"[The 25th National Security Advisor, Michael F...",[1],"[[0, 1]]","[[True, True]]",[0],1.0,1.0,1.0,0.692493,NaN
4,1,5a761900554299109176e648,mixtral-8x7b-instruct-v0.1.Q8_0,1,0.1,t5_xxl_true_nli_mixture,mistral-7b-instruct-v0.2.Q8_0,1.000000,1.0,1,...,"[The 25th National Security Advisor, Michael F...",[1],"[[0, 1]]","[[True, True]]",[0],1.0,1.0,1.0,0.689531,NaN


In [3]:
results = results.drop(columns=["id"])
all_obj_cols = results.select_dtypes(include=["object"]).columns
drop_obj_cols = list(set(all_obj_cols) - set(params_names))
results = results.drop(columns=drop_obj_cols)

In [4]:
gb = results.groupby(params_names)
mean = gb.mean()
mean = mean.drop(columns=["question_idx"])
mean["n_examples"] = gb.size()
if mean["n_examples"].nunique() != 1:
    print("Warning: not all rows have the same number of examples")
mean

citations/ais_recall  \
llm                             prompt_id temperature nli                     nlg                                                   
gpt-3.5-turbo-0125              1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0              0.729222   
mistral-7b-instruct-v0.2.Q8_0   1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0              0.549818   
                                2         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0              0.488163   
                                3         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0              0.512314   
mixtral-8x7b-instruct-v0.1.Q8_0 1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0              0.637489   

                                                                                                             citations/ais_precision  \
llm                             prompt_id temperature nli                     nlg                                                      
gpt-3.5-turbo-0125              1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                 0.991389   
mistral-7b-instruct-v0.2.Q8_0   1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                 0.876011   
                                2         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                 0.843323   
                                3         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                 0.877921   
mixtral-8x7b-instruct-v0.1.Q8_0 1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                 0.918824   

                                                                                                             citations/n_sentences  \
llm                             prompt_id temperature nli                     nlg                                                    
gpt-3.5-turbo-0125              1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0               2.020000   
mistral-7b-instruct-v0.2.Q8_0   1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0               2.853333   
                                2         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0               3.190000   
                                3         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0               2.420000   
mixtral-8x7b-instruct-v0.1.Q8_0 1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0               2.647651   

                                                                                                             citations/n_total_citations  \
llm                             prompt_id temperature nli                     nlg                                                          
gpt-3.5-turbo-0125              1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                     2.143333   
mistral-7b-instruct-v0.2.Q8_0   1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                     3.203333   
                                2         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                     4.113333   
                                3         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                     3.133333   
mixtral-8x7b-instruct-v0.1.Q8_0 1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                     2.991051   

                                                                                                             citations/n_correct_citations  \
llm                             prompt_id temperature nli                     nlg                                                            
gpt-3.5-turbo-0125  